# Ячейка 1. Импорты и адаптивная отрисовка матриц

In [ ]:

from __future__ import annotations
from typing import List, Tuple, Dict, Any
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def plot_matrix(M: np.ndarray, title: str = "", *, cell_scale: float = 0.6,
                show_values: bool = True, annotate_threshold: int = 24) -> None:
    """
    Визуализация матрицы кодирования или вспомогательной матрицы, возникающей
    в задачах теории кодирования (например, матриц Джекобстала, Адамара,
    бинаризованных кодовых матриц и т.п.).

    Функция строит графическое представление матрицы в виде таблицы цветов,
    что позволяет удобно анализировать структуру строк, симметрию,
    ортогональность, циклические свойства и другие характеристики,
    важные при исследовании кодов Хэмминга, кодов Адамара, кодов с малой
    корреляцией и других классов бинарных и ±1-матриц.

    Параметры
    ----------
    M : np.ndarray
        Входная матрица размера (n × m), содержащая значения элементов
        кодовой структуры. Это может быть бинарная матрица (0/1),
        матрица в алфавите {+1, -1}, или произвольная числовая матрица,
        полученная после преобразований (например, применения алгоритма
        Джекобстала).

    title : str, optional
        Заголовок для графика. Используется для пояснений при анализе
        нескольких матриц (например, J, H и H_bin в цепочке построения
        матрицы Адамара).

    cell_scale : float, optional
        Масштаб ячейки. Управляет итоговым размером изображения,
        чтобы матрица оставалась читаемой как при малых, так и при больших
        размерах.

    show_values : bool, optional
        Если True, то внутри ячеек выводятся численные значения матрицы.
        Полезно для небольших матриц (например, q ≤ 23), когда важно
        видеть точные значения ±1, 0, 1.

    annotate_threshold : int, optional
        Максимальный размер измерения матрицы, при котором допускается
        вывод значений внутри ячеек. Это предотвращает перегрузку графика
        текстом при больших q.

    Возвращаемое значение
    ---------------------
    None
        Функция выполняет только визуализацию, не изменяя исходную матрицу
        и не возвращая данных.

    Описание алгоритма
    -------------------
    1. Рассчитывается оптимальный размер итоговой фигуры на основе числа
       строк и столбцов, чтобы сохранить пропорции и читаемость.

    2. Матрица отображается функцией `plt.imshow`, где значения кодовой
       структуры визуализируются как цветовые ячейки. Применяется светлая
       пастельная colormap, удобная для анализа структуры без сильной
       контрастности.

    3. Добавляется координатная сетка для удобства чтения строк и столбцов.
       Это делает видимыми границы блоков, циклические сдвиги и другие
       закономерности.

    4. Если матрица небольшая, внутри ячеек дополнительно выводятся
       численные значения для точного анализа.

    5. График аккуратно упаковывается и отображается пользователю.
    """

    nrows, ncols = M.shape
    cell = max(0.25, min(1.2, float(cell_scale)))
    width  = max(4.0, min(16.0, ncols * cell))
    height = max(4.0, min(16.0, nrows * cell))

    plt.figure(figsize=(width, height))
    plt.imshow(M, interpolation="nearest", cmap="Pastel2")
    plt.title(title)
    plt.xlabel("Столбец")
    plt.ylabel("Строка")

    # Решётка
    plt.xticks(range(ncols))
    plt.yticks(range(nrows))
    ax = plt.gca()
    ax.set_xticks([x-0.5 for x in range(1, ncols)], minor=True)
    ax.set_yticks([y-0.5 for y in range(1, nrows)], minor=True)
    ax.grid(which="major", linestyle="-", linewidth=0.3, alpha=0.5)
    ax.grid(which="minor", linestyle="-", linewidth=0.6, alpha=0.6)

    effective_show = bool(show_values) and max(nrows, ncols) <= annotate_threshold
    if effective_show:
        base = 14.0
        fs = max(6.0, base - 0.3 * max(nrows, ncols))
        for i in range(nrows):
            for j in range(ncols):
                val = M[i, j]
                try:
                    f = float(val)
                    s = str(int(f)) if f.is_integer() else f"{f:.2f}"
                except Exception:
                    s = str(val)
                plt.text(j, i, s, ha="center", va="center", fontsize=fs)

    plt.tight_layout()
    plt.show()


## Ячейка 2. Подалгоритм: символ Лежандра с полным трассированием

In [ ]:

def legendre_symbol_trace(a: int, p: int) -> Tuple[int, List[Dict[str, Any]]]:
    """
    Вычисление символа Лежандра (a/p) с подробной трассировкой всех шагов
    алгоритма квадратичной взаимности.

    Функция реализует классический алгоритм вычисления символа Лежандра,
    основанный на:
        • приведении по модулю p,
        • обработке знака,
        • правилах для фактора 2,
        • законе квадратичной взаимности,
        • итеративной замене пары (a, p) на (p mod a, a).

    Одновременно формируется подробный журнал действий, фиксирующий каждое
    преобразование (изменение знака, редукцию, применённые законы,
    проверку условий чётности), что делает функцию удобной как для учебных
    целей, так и для анализа корректности построения первых строк матрицы
    Джекобстала в задачах теории кодирования и построения матриц Адамара.

    Параметры
    ----------
    a : int
        Число, для которого вычисляется символ Лежандра (a/p). Может быть
        отрицательным или произвольным целым; алгоритм корректно выполняет
        редукцию по модулю p.

    p : int
        Нечётное простое число. Модуль, относительно которого вычисляется
        символ Лежандра. Предполагается, что p является простым, поскольку
        свойства квадратичной взаимности используются в полной форме.

    Возвращаемое значение
    ---------------------
    Tuple[int, List[Dict[str, Any]]]
        Первый элемент:
            •  1 — если a является квадратичным вычетом по модулю p,
            • -1 — если a является невычетом,
            •  0 — если a ≡ 0 (mod p).

        Второй элемент — список шагов трассировки, где каждый шаг
        представлен словарём с полями:
            • "шаг"         — описание выполняемого действия,
            • "a"           — текущее значение a,
            • "p"           — текущее значение p,
            • "знак"        — текущий множитель s ∈ {+1, -1},
            • "примечание"  — дополнительное пояснение (например,
                               "p ≡ 3 (mod 4)" или информация о чётности).

    Описание алгоритма
    -------------------
    Алгоритм следует стандартным преобразованиям символа Лежандра:

    1. Приведение `a` по модулю `p`.
       Если a ≡ 0 (mod p), результат равен 0.

    2. Обработка отрицательного `a`.
       Используется закон для (-1/p):
            (-1/p) = (-1)^((p-1)/2).

    3. Факторизация чётных частей a.
       При каждом делении на 2 используется формула:
            (2/p) = (-1)^((p^2 - 1)/8).

    4. Применение закона квадратичной взаимности:
            (a/p)(p/a) = (-1)^(((a-1)/2)*((p-1)/2)),
       что приводит к обмену ролей (a, p) → (p mod a, a), иногда со сменой знака.

    5. Цикл продолжается до достижения a = 1, при котором (1/p) = 1,
       и результат равен текущему multiplicator s.
    """

    trace: List[Dict[str, Any]] = []
    s: int = 1

    def log(step: str, a_val: int, p_val: int, note: str = "") -> None:
        trace.append({"шаг": step, "a": a_val, "p": p_val, "знак": s, "примечание": note})

    a0 = a
    a = a % p
    if a == 0:
        log("Редукция: a ≡ 0 (mod p) ⇒ (a/p)=0", a, p, f"Исходное a={a0}")
        return 0, trace
    log("Редукция: a ← a mod p", a, p, f"Исходное a={a0}")

    if a < 0:
        a = -a
        if ((p - 1) // 2) % 2 == 1:
            s *= -1
            log("Знак: применено (-1/p) = -1", a, p, "p ≡ 3 (mod 4)")
        else:
            log("Знак: применено (-1/p) = +1", a, p, "p ≡ 1 (mod 4)")

    while a % 2 == 0:
        a //= 2
        exp = ((p * p - 1) // 8) % 2
        if exp == 1:
            s *= -1
            log("Фактор 2: (2/p) = -1", a, p, "(p^2-1)/8 нечётно")
        else:
            log("Фактор 2: (2/p) = +1", a, p, "(p^2-1)/8 чётно")

    if a == 1:
        log("Тривиальный случай: (1/p)=1", a, p)
        return s, trace

    while a != 1:
        power = ((a - 1) // 2) * ((p - 1) // 2)
        if power % 2 == 1:
            s *= -1
            log("Взаимность: множитель -1", a, p, "Показатель нечётен")
        else:
            log("Взаимность: множитель +1", a, p, "Показатель чётен")

        a, p = p % a, a
        log("Смена ролей и редукция: (p mod a, a)", a, p)

        if a == 0:
            log("Редукция: a ≡ 0 (mod p) ⇒ (a/p)=0", a, p)
            return 0, trace

        if a < 0:
            a = -a
            if ((p - 1) // 2) % 2 == 1:
                s *= -1
                log("Знак: применено (-1/p) = -1", a, p, "p ≡ 3 (mod 4)")
            else:
                log("Знак: применено (-1/p) = +1", a, p, "p ≡ 1 (mod 4)")

        while a % 2 == 0:
            a //= 2
            exp = ((p * p - 1) // 8) % 2
            if exp == 1:
                s *= -1
                log("Фактор 2: (2/p) = -1", a, p, "(p^2-1)/8 нечётно")
            else:
                log("Фактор 2: (2/p) = +1", a, p, "(p^2-1)/8 чётно")

        if a == 1:
            log("Тривиальный случай: (1/p)=1", a, p)
            break

    return s, trace


## Ячейка 3. Матрица Джекобстола: первая строка и цикличность

In [ ]:
def jacobsthal_first_row(q: int) -> Tuple[np.ndarray, List[Dict[str, Any]]]:
    """
    Формирование первой строки r^(0) матрицы Джекобстала порядка q
    с одновременной трассировкой вычисления символов Лежандра.

    Первая строка r^(0) матрицы Джекобстала определяется формулой:
        r^(0)_k = (k / q),  где (·/q) — символ Лежандра.
    При этом по определению r^(0)_0 = 0.

    Данная функция вычисляет значения символов Лежандра для всех k = 0, 1, …, q−1
    и возвращает:
        • массив первой строки из значений {0, +1, −1},
        • список структурированных отчётов о вычислениях для каждого k,
        включая подробную трассировку алгоритма квадратичной взаимности.

    Параметры
    ----------
    q : int
        Простое число, удовлетворяющее условию q ≡ 3 (mod 4) при построении
        матрицы Адамара по схеме Джекобстала. Предполагается, что q — нечётное
        простое, поскольку вычисление символа Лежандра требует этого условия.

    Возвращаемое значение
    ---------------------
    Tuple[np.ndarray, List[Dict[str, Any]]]
        Первый элемент — numpy-массив длины q, содержащий первую строку
        r^(0) матрицы Джекобстала:
            r^(0)_0 = 0,
            r^(0)_k = (k/q) ∈ {+1, −1} для k > 0.

        Второй элемент — список длины q, где каждый элемент представляет собой
        словарь:
            {
            "k": k,                  # текущий индекс
            "value": (k/q),          # вычисленное значение символа Лежандра
            "trace": [...шаги...]    # список шагов трассировки legendre_symbol_trace
            }

    Описание алгоритма
    -------------------
    1. Для k = 0 значение символа Лежандра равно 0, что фиксируется вручную.

    2. Для каждого последующего k вызывается алгоритм legendre_symbol_trace(k, q),
        который:
            • приводит аргумент по модулю q,
            • обрабатывает знак,
            • обрабатывает фактор 2,
            • применяет закон квадратичной взаимности,
            • итеративно заменяет (a, p) → (p mod a, a),
            • возвращает символ Лежандра и подробный набор шагов.

    3. Все значения собираются в список, формируя первую строку r^(0).
    """
    row_vals: List[int] = []
    steps: List[Dict[str, Any]] = []
    for k in range(q):
        if k == 0:
            row_vals.append(0)
            steps.append({"k": k, "value": 0, "trace": [{"шаг": "Определение", "a": 0, "p": q, "знак": 1, "примечание": "(0/q)=0"}]})
            continue
        val, tr = legendre_symbol_trace(k, q)
        row_vals.append(val)
        steps.append({"k": k, "value": val, "trace": tr})
    return np.array(row_vals, dtype=int), steps


def jacobsthal_matrix_from_first_row(first_row: np.ndarray) -> np.ndarray:
    """
    Формирование матрицы Джекобстала J порядка q по заданной первой строке r^(0).

    Матрица Джекобстала J имеет размер q × q и обладает ключевым свойством:
        каждая строка получается циклическим сдвигом предыдущей на один элемент вправо.

    Если r^(0) — первая строка, то
        r^(i) = shift(r^(i-1))  для всех i = 1, …, q−1.

    Такая структура позволяет использовать матрицу Джекобстала при построении
    матриц Адамара порядка q+1, обеспечивая ортогональность строк и лёгкость
    вычисления преобразований.

    Параметры
    ----------
    first_row : np.ndarray
        Первая строка r^(0) матрицы Джекобстала. Должна быть одномерным
        массивом длины q и содержать элементы {0, +1, −1}.

    Возвращаемое значение
    ---------------------
    np.ndarray
        Квадратная матрица J размера q × q, где первая строка совпадает
        с `first_row`, а каждая следующая строка является её циклическим
        сдвигом.

    Описание алгоритма
    -------------------
    1. Создаётся матрица J из нулей размера q × q.
    2. В первую строку записывается переданный вектор first_row.
    3. Для каждой следующей строки выполняется циклический сдвиг предыдущей:
           J[i] = roll(J[i−1], 1)
    4. Итоговая матрица сохраняет важное свойство:
           строки являются эквивалентными по циклическим сдвигам.
    """
    q = first_row.size
    J = np.zeros((q, q), dtype=int)
    J[0, :] = first_row
    for i in range(1, q):
        J[i, :] = np.roll(J[i-1, :], 1)
    return J


## Ячейка 4. Матрица Адамара и три кода Адамара

In [ ]:

def hadamard_from_jacobsthal(J: np.ndarray) -> np.ndarray:
    """
    Построение матрицы Адамара H порядка q+1 по матрице Джекобстала J.

    В соответствии с классической конструкцией Джекобстала–Адамара,
    матрица H имеет размер (q+1) × (q+1) и формируется следующим образом:

        • первая строка и первый столбец полностью состоят из +1;
        • подматрица H[1:, 1:] получается из J после инверсии диагональных элементов;
        • диагональные элементы матрицы J заменяются на −1, что обеспечивает
          ортогональность строк матрицы H.

    Данная конструкция корректна для простых q, удовлетворяющих условию q ≡ 3 (mod 4).

    Параметры
    ----------
    J : np.ndarray
        Матрица Джекобстала размера q × q с элементами {0, +1, −1}.
        Используется для формирования внутренней части матрицы Адамара.

    Возвращаемое значение
    ---------------------
    np.ndarray
        Матрица Адамара H размера (q+1) × (q+1) с элементами {+1, −1}.
        Матрица обладает ортогональностью строк и столбцов и может использоваться
        для построения кодов Адамара и производных кодовых конструкций.

    Описание алгоритма
    -------------------
    1. Создаётся матрица H из единиц размера (q+1) × (q+1).
    2. Копируется матрица Джекобстала в B.
    3. Диагональные элементы B[i, i] заменяются на −1.
    4. Блок H[1:, 1:] заменяется на преобразованную матрицу B.
    5. Первая строка и первый столбец остаются заполненными единицами.
    """
    q = J.shape[0]
    n = q + 1
    H = np.ones((n, n), dtype=int)
    B = J.copy()
    for i in range(q):
        B[i, i] = -1
    H[1:, 1:] = B
    return H

def to_binary_pm(M: np.ndarray) -> np.ndarray:
    """
    Преобразование матрицы с элементами {+1, −1} в бинарную матрицу {0, 1}.

    Функция выполняет стандартное преобразование:
        −1  →  1
        +1  →  0

    Такое кодирование используется при переходе от матрицы Адамара H
    к бинарному представлению, необходимому при построении линейных кодов.

    Параметры
    ----------
    M : np.ndarray
        Матрица, элементы которой равны +1 или −1.

    Возвращаемое значение
    ---------------------
    np.ndarray
        Бинарная матрица того же размера, содержащая значения {0, 1},
        где 1 соответствует элементу −1 исходной матрицы.
    """
    return (M == -1).astype(int)

def hadamard_codes_from_H(H_bin: np.ndarray) -> Dict[str, np.ndarray]:
    """
    Построение трёх бинарных кодов на основе бинарной матрицы Адамара H_bin.

    На основе стандартной конструкции кодов Адамара формируются три разновидности:

        1. C1 — код параметров (n−1, n, n/2):
           матрица из строк H_bin без первого столбца.

        2. C2 — код параметров (n−1, 2n, n/2 − 1):
           объединение матрицы C1 и её инверсии 1 − C1.

        3. C3 — код параметров (n, 2n, n/2):
           объединение H_bin и его инверсии 1 − H_bin.

    Эти конструкции используются при построении кодов с хорошими
    корреляционными и разделительными свойствами.

    Параметры
    ----------
    H_bin : np.ndarray
        Бинарная матрица Адамара размера n × n (значения 0/1).

    Возвращаемое значение
    ---------------------
    Dict[str, np.ndarray]
        Словарь, где ключи — обозначения кодов, а значения — соответствующие
        кодовые матрицы:
            {
                "C1_(n-1,n,n/2)":  C1,
                "C2_(n-1,2n,n/2-1)": C2,
                "C3_(n,2n,n/2)":   C3
            }

    Описание алгоритма
    -------------------
    1. Из H_bin удаляется первый столбец → формируется C1.
    2. C2 получается вертикальным объединением C1 и его дополнения (1 − C1).
    3. C3 строится аналогично, но на базе всей H_bin.
    4. Все кодовые матрицы возвращаются в словаре.
    """
    n = H_bin.shape[0]
    C1 = H_bin[:, 1:].copy()
    C2 = np.vstack([C1, 1 - C1])
    C3 = np.vstack([H_bin, 1 - H_bin])
    return {
        "C1_(n-1,n,n/2)": C1,
        "C2_(n-1,2n,n/2-1)": C2,
        "C3_(n,2n,n/2)": C3,
    }

def min_hamming_distance(code: np.ndarray) -> int:
    """
    Вычисление минимального кодового расстояния d_min между строками кода.

    Минимальное расстояние Хэмминга — ключевая характеристика линейного или
    нелинейного кода, определяющая:
        • число ошибок, которые можно гарантированно обнаружить,
        • число ошибок, которые можно исправить,
        • качество и надёжность кода.

    Функция перебирает все попарные расстояния между строками кодовой матрицы
    и определяет минимальное из них.

    Параметры
    ----------
    code : np.ndarray
        Матрица размера M × L, где каждая строка — кодовое слово длины L.
        Элементы предполагаются бинарными (0/1).

    Возвращаемое значение
    ---------------------
    int
        Минимальное расстояние Хэмминга между двумя различными кодовыми словами.
        Значение принимает диапазон [0, L].

    Описание алгоритма
    -------------------
    1. Инициализация d_min = L.
    2. Для всех пар строк (i, j) вычисляется XOR и его вес Хэмминга.
    3. Если расстояние меньше текущего d_min — обновляем его.
    4. Возвращаем минимальное найденное расстояние.
    """
    M, L = code.shape
    dmin = L
    for i in range(M):
        for j in range(i+1, M):
            d = int(np.count_nonzero(code[i] ^ code[j]))
            if d < dmin:
                dmin = d
    return int(dmin)


## Итоговая ячейка. Сквозной запуск + интерактивные настройки масштаба

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

def _code_to_dataframe(C: np.ndarray) -> pd.DataFrame:
    """
    Преобразование кодовой матрицы в удобное табличное представление.

    Каждая строка бинарной матрицы кода преобразуется в:
        • индекс строки,
        • строковое представление битов,
        • вес Хэмминга (число единиц).

    Такое табличное представление удобно для анализа структуры кода,
    сравнения кодовых слов, проверки равновесности строк и визуального
    изучения построенного кодового семейства.

    Параметры
    ----------
    C : np.ndarray
        Бинарная матрица размера M × L, где каждая строка — кодовое слово длины L.

    Возвращаемое значение
    ---------------------
    pd.DataFrame
        Таблица из трёх колонок:
            • "index" — номер кодового слова,
            • "bits" — строковое представление битовой последовательности,
            • "weight" — вес Хэмминга данного слова.
    """
    M, L = C.shape
    rows = []
    for i in range(M):
        bits = "".join(str(int(b)) for b in C[i])
        wt = int(np.count_nonzero(C[i]))
        rows.append({"index": i, "bits": bits, "weight": wt})
    return pd.DataFrame(rows, columns=["index", "bits", "weight"])

def run_pipeline(q: int = 7, show_trace: bool = True, *, cell_scale: float = 0.6, show_values: bool = True, annotate_threshold: int = 40) -> None:
    """
    Полный конвейер построения матриц Джекобстала и Адамара,
    их бинаризации и генерации кодов Адамара с последующим выводом
    минимального расстояния.

    Функция объединяет все этапы алгоритма в единую последовательность:

        1. Вычисление первой строки r^(0) матрицы Джекобстала
           с использованием символа Лежандра.

        2. Построение матрицы Джекобстала J.

        3. Построение матрицы Адамара H порядка q+1.

        4. Преобразование H в бинарную матрицу H_bin.

        5. Генерация трёх кодов Адамара (C1, C2, C3)
           с использованием стандартных конструкций.

        6. Вывод минимального кодового расстояния d_min
           и табличного представления кода.

        7. При необходимости вывод подробной трассировки вычислений
           символов Лежандра.

    Параметры
    ----------
    q : int
        Простое число, удовлетворяющее q ≡ 3 (mod 4). Определяет порядок
        матриц Джекобстала и Адамара.

    show_trace : bool
        Если True — печатает пошаговую трассировку вычисления символов Лежандра.

    cell_scale : float
        Коэффициент масштабирования ячеек при визуализации матриц.

    show_values : bool
        Если True — отображает численные значения внутри матриц
        при построении графиков.

    annotate_threshold : int
        Максимальный размер измерения матрицы, при котором подписи значений
        внутри ячеек остаются включёнными.

    Возвращаемое значение
    ---------------------
    None
        Вся работа выполняется «в сторону»: печать, графики и таблицы.
    """
    row, steps = jacobsthal_first_row(q)
    print(f"q = {q} (q ≡ 3 mod 4). Первая строка r^(0):")
    print(row.tolist())

    if show_trace:
        for rec in steps:
            k = rec["k"]
            val = rec["value"]
            print(f"\n--- k = {k}, (k/q) = {val} ---")
            for t in rec["trace"]:
                print(f"  {t['шаг']}: a={t['a']}, p={t['p']}, знак={t['знак']}; {t.get('примечание','')}")

    J = jacobsthal_matrix_from_first_row(row)
    plot_matrix(J, title="Матрица Джекобстола J (значения в {0, +1, -1})", cell_scale=cell_scale, show_values=show_values, annotate_threshold=annotate_threshold)

    H = hadamard_from_jacobsthal(J)
    plot_matrix(H, title="Матрица Адамара H (значения в {+1, -1})", cell_scale=cell_scale, show_values=show_values, annotate_threshold=annotate_threshold)

    H_bin = to_binary_pm(H)
    plot_matrix(H_bin, title="Бинарная матрица H (0/1)", cell_scale=cell_scale, show_values=show_values, annotate_threshold=annotate_threshold)

    codes = hadamard_codes_from_H(H_bin)
    for name, C in codes.items():
        dmin = min_hamming_distance(C)
        print(f"\n{name}: форма {C.shape}, d_min = {dmin}")
        display(_code_to_dataframe(C))

def _is_prime(n: int) -> bool:
    """
    Простейшая проверка числа на простоту методом перебора делителей.

    Используется для проверки допустимости выбора параметра q при построении
    матриц Джекобстала и Адамара, которые требуют простого q.

    Параметры
    ----------
    n : int
        Проверяемое число.

    Возвращаемое значение
    ---------------------
    bool
        True, если n — простое число,
        False — иначе.
    """
    if n < 2:
        return False
    if n % 2 == 0:
        return n == 2
    i = 3
    while i * i <= n:
        if n % i == 0:
            return False
        i += 2
    return True

try:
    from ipywidgets import interact, IntSlider, Checkbox, FloatSlider, HTML
    def _ui(q: int = 7, show_trace: bool = True, show_values: bool = True, cell_scale: float = 0.6):
        msgs = []
        if q % 4 != 3:
            msgs.append(f"q ≡ {q % 4} (mod 4), требуется q ≡ 3 (mod 4).")
        if not _is_prime(q):
            msgs.append("q не простое.")
        if msgs:
            display(HTML("<b style='color:#b00'>Невалидный выбор q:</b> " + " ".join(msgs)))
            return
        run_pipeline(q=q, show_trace=show_trace, cell_scale=cell_scale, show_values=show_values)

    interact(_ui,
             q=IntSlider(min=3, max=99, step=4, value=7, description='q'),
             show_trace=Checkbox(value=True, description='Показывать трассировку'),
             show_values=Checkbox(value=True, description='Подписи в ячейках'),
             cell_scale=FloatSlider(value=0.6, min=0.3, max=1.0, step=0.05, description='Масштаб ячеек'))
except Exception as e:
    print("ipywidgets недоступны — выполняю дефолтный запуск: run_pipeline(q=7, show_trace=True)")
    run_pipeline(q=7, show_trace=True, cell_scale=0.6, show_values=True)
